In [62]:
import matplotlib.animation as animation

from brian2 import *


### global parameters
defaultclock.dt = 15*ms


### simulation code
def run_sim(N, K, F, Delta, p1, p2, omega_0, tau, sigma, duration, random_seed=214040893):
    seed(random_seed)

    eqs = '''
    dTheta/dt = omega + K/N*coupling + noise: 1
    omega : Hz (constant) # intrinsic frequency
    coupling : 1
    noise : Hz (constant)
    '''

    oscillators = NeuronGroup(N, eqs, method='euler')
    oscillators.Theta = np.random.rand(N)  # same initial phase angle

    
    spike_probability = 0.2 
    spike_magnitude = 5 
    impulse_noise = (np.random.rand(N) < spike_probability) * np.random.uniform(-spike_magnitude, spike_magnitude, N)
    oscillators.noise =  impulse_noise * Hz


    #CHANGE THIS TO ANOTHER VALUE
    lorentzian_omegas = omega_0 + Delta * np.tan(np.pi * (np.random.rand(N) - 0.5))
    #lorentzian_omegas = omega_0
    oscillators.omega = lorentzian_omegas


    connections = Synapses(oscillators, oscillators,
                        'coupling_post = sin(Theta_pre - Theta_post) : 1 (summed)')
    connections.connect()  # all-to-all

    mon = StateMonitor(oscillators, 'Theta', record=True)

   # Variable to hold the current value of z
    z_values = []
    @network_operation()
    def compute_z():
        # Calculate z(t) using the current phases
        z = np.mean(np.exp(1j * oscillators.Theta))
        z_values.append(z)

    net = Network(oscillators, connections, mon, compute_z)
    net.run(duration)

    return z_values, oscillators.Theta
        

In [63]:
initial_values = [[10.0, 3.5, 1.4], [4.5, 3.5, 1.4], [20, 2, 5]]
sigma = 2*pi/(24)*Hz
Delta = sigma
for count, arr in enumerate(initial_values): 
    K = arr[0] * Delta      # Coupling strength
    F = arr[1] * Delta # Amplitude of external driving force
    Omega = arr[2] * Delta
    omega_0 = sigma - Omega
    colors = ['red']  # Colors for each travel point
    labels = ['9.5h eastward']
    tau = 0*second  # Time of travel
    #plt.figure(figsize=(4, 4))
    real_p2 = 9.5 * (2 * pi) / 24
    for I in np.arange(1.1, 30.1, 0.5):
        K  = I * Hz
        z_final, theta_values = run_sim(50, K, F, Delta, 0, real_p2, omega_0, tau, sigma, 5*second)
        #print(f'z_value = {abs(z_final[-1])}\n')
        if(abs(z_final[-1]) > 0.99) :
            print(f'Synchronized at {I}*Hz for initial_value at {count}')
            break
        z_real = np.real(z_final)
        z_imag = np.imag(z_final)
        #plt.plot(z_real, z_imag, label=labels[i], color=colors[i])
    # Labels and formatting
    # plt.xlabel("Re(z)")
    # plt.ylabel("Im(z)")
    # plt.title(f"Panel: {count}")
    # plt.legend()
    # plt.axis("equal")
    # plt.grid()
# plt.show()

Synchronized at 20.1*Hz for initial_value at 0
Synchronized at 20.1*Hz for initial_value at 1
Synchronized at 20.1*Hz for initial_value at 2


In [ ]:
#Types of noises for refrence 

#constant/normal noise
    #noise = 0
    #noise = 2

#Random noise
    #noise = np.random.rand(N)


#gaussian noise
    # gaussian_noise = np.random.normal(0, 1, N)
    # gaussian_noise = np.random.normal(5, 1, N)

#non-gaussian noise (impulses)
    # spike_probability = 0.2 
    # spike_magnitude = 5 
    # impulse_noise = (np.random.rand(N) < spike_probability) * np.random.uniform(-spike_magnitude, spike_magnitude, N)

#non-gaussian noise (Poisson Noise)
    # lambda_param = 3  # Average rate of spikes
    # poisson_noise = np.random.poisson(lambda_param, N) - lambda_param  # Center at 0